In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/TFG/Pruebas ejecucion

/content/drive/.shortcut-targets-by-id/1SmZYQ5FtbDJftz-DFi07WmU23UAjYJGL/TFG/Pruebas ejecucion


In [ ]:
!apt install --allow-change-held-packages libcudnn8=8.7.0.84-1+cuda11.8
!pip install tensorflow keras tensorflow-estimator tensorflow-text

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libcudnn8 is already the newest version (8.7.0.84-1+cuda11.8).
0 upgraded, 0 newly installed, 0 to remove and 23 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [ ]:
!pip install protobuf==3.20.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip show protobuf

Name: protobuf
Version: 3.20.3
Summary: Protocol Buffers
Home-page: https://developers.google.com/protocol-buffers/
Author: 
Author-email: 
License: BSD-3-Clause
Location: /usr/local/lib/python3.10/dist-packages
Requires: 
Required-by: google-api-core, google-cloud-bigquery, google-cloud-bigquery-storage, google-cloud-datastore, google-cloud-firestore, google-cloud-language, google-cloud-translate, googleapis-common-protos, grpcio-status, proto-plus, tensorboard, tensorflow, tensorflow-datasets, tensorflow-hub, tensorflow-metadata


In [ ]:
# !apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2
# !pip uninstall -y -q tensorflow keras tensorflow-estimator tensorflow-text
# !pip install protobuf==3.20.3
# !pip install -q -U tensorflow-text==2.11.0 tensorflow==2.11.0

In [ ]:
from data_preprocessing import create_datasets, text_vectorization, process_text
from model import Transformer, CustomSchedule, masked_loss, masked_acc

import tensorflow as tf
from tensorflow import keras

import pandas as pd
from timeit import default_timer as timer

## Support functions

In [ ]:
train, val, test = create_datasets()
context_text_processor, target_text_processor = text_vectorization(train)

# print(context_text_processor.get_vocabulary()[:10])
# print(target_text_processor.get_vocabulary()[:10])

train_ds = train.map(lambda x, y: process_text(x, y, context_text_processor, target_text_processor), tf.data.AUTOTUNE)
val_ds = val.map(lambda x, y: process_text(x, y, context_text_processor, target_text_processor), tf.data.AUTOTUNE)
test_ds = val.map(lambda x, y: process_text(x, y, context_text_processor, target_text_processor), tf.data.AUTOTUNE)

In [ ]:
def create_metrics(history, timing_callback, file_name):
    try:
        metrics = pd.read_csv(file_name)
    except:
        metrics = pd.DataFrame(columns = ['loss', 'masked_acc', 'val_loss', 'val_masked_acc', 'training_time'])

    new_metrics = pd.DataFrame(history.history)
    times = timing_callback  # Extraemos los valores numéricos de los tiempos de entrenamiento
    new_metrics['training_time'] = times  # Agregamos la columna 'training_time' con los valores de 'times'
    metrics = metrics.append(new_metrics, ignore_index=True)

    metrics.to_csv('metrics/'+file_name+'.csv', index = False)

In [ ]:
class TimingCallback(keras.callbacks.Callback):
    def __init__(self, logs={}):
        self.logs=[]
    def on_epoch_begin(self, epoch, logs={}):
        self.starttime = timer()
    def on_epoch_end(self, epoch, logs={}):
        self.logs.append(timer()-self.starttime)

cb = TimingCallback()

## Model

In [ ]:
#! Model
num_layers = 6
d_model = 256
dff = 512
num_heads = 8
dropout_rate = 0.3

In [ ]:
MODEL_NAME = 'en_es_2_' + str(num_layers) + '_' + str(d_model) + '_' + str(dff) + '_' + str(num_heads) + '_' + str(dropout_rate).replace('.','-')

In [ ]:
transformer = Transformer(
    num_layers = num_layers,
    d_model = d_model,
    num_heads = num_heads,
    dff = dff,
    input_vocab_size = context_text_processor.vocabulary_size(),
    target_vocab_size = target_text_processor.vocabulary_size(),
    dropout_rate = dropout_rate,
)

## Training

In [ ]:
#! Loss and Optimizer
learning_rate = CustomSchedule(d_model)
my_optimizer = tf.keras.optimizers.legacy.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

In [ ]:
#! Compile
transformer.compile(
    loss = masked_loss,
    optimizer=my_optimizer,
    metrics = [masked_acc],
)

In [ ]:
history = transformer.fit(
    train_ds,
    epochs = 10,
    validation_data = val_ds,
    callbacks=[cb],
)

Epoch 1/10
820/820 [==============================] - 115s 140ms/step - loss: 2.6091 - masked_acc: 0.5333 - val_loss: 3.0546 - val_masked_acc: 0.4953
Epoch 2/10
820/820 [==============================] - 115s 140ms/step - loss: 2.4926 - masked_acc: 0.5484 - val_loss: 2.8980 - val_masked_acc: 0.5161
Epoch 3/10
820/820 [==============================] - 114s 139ms/step - loss: 2.3807 - masked_acc: 0.5626 - val_loss: 2.8901 - val_masked_acc: 0.5201
Epoch 4/10
820/820 [==============================] - 114s 139ms/step - loss: 2.2844 - masked_acc: 0.5764 - val_loss: 2.8782 - val_masked_acc: 0.5239
Epoch 5/10
820/820 [==============================] - 114s 139ms/step - loss: 2.2086 - masked_acc: 0.5865 - val_loss: 2.8838 - val_masked_acc: 0.5275
Epoch 6/10
820/820 [==============================] - 114s 139ms/step - loss: 2.1352 - masked_acc: 0.5975 - val_loss: 2.8669 - val_masked_acc: 0.5288
Epoch 7/10
820/820 [==============================] - 115s 140ms/step - loss: 2.0842 - masked_acc: 0

In [ ]:
print(cb.logs)
print(sum(cb.logs))

[136.27683706300013, 114.87772929500034, 114.77037471299991, 114.4568885220001, 115.02371567, 114.32720948399992, 114.51269118899972, 114.82905802699997, 114.68911488799995, 114.73681123799997, 114.96024260000013, 115.04256749000069, 114.40339059799953, 114.27756443599992, 114.28743367499919, 114.26483838799959, 114.62523914899975, 114.35008968999955, 114.04654443099935, 114.55699829200057]
2313.3153388379983


In [ ]:
times = cb.logs[-10:]

In [ ]:
times

[114.96024260000013,
 115.04256749000069,
 114.40339059799953,
 114.27756443599992,
 114.28743367499919,
 114.26483838799959,
 114.62523914899975,
 114.35008968999955,
 114.04654443099935,
 114.55699829200057]

In [ ]:
create_metrics(history, times, MODEL_NAME)

<ipython-input-39-fffef7b59abb>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics = metrics.append(new_metrics, ignore_index=True)


In [ ]:
transformer.save_weights('models/' + MODEL_NAME +  '.h5')

In [ ]:
# Save model
transformer.save('modelsFull/' + MODEL_NAME)

## Reload model

In [ ]:
new_model = Transformer(
    num_layers = num_layers,
    d_model = d_model,
    num_heads = num_heads,
    dff = dff,
    input_vocab_size = context_text_processor.vocabulary_size(),
    target_vocab_size = target_text_processor.vocabulary_size(),
    dropout_rate = dropout_rate,
)

new_model.compile(
    loss = masked_loss,
    optimizer=my_optimizer,
    metrics = [masked_acc],
)

# Llama al modelo con datos ficticios para construirlo
dummy_input = tf.ones((1, 1), dtype=tf.int32)
dummy_context = tf.ones((1, 1), dtype=tf.int32)
new_model((dummy_context, dummy_input), training=False)

# Construye el optimizador con las variables del modelo
# my_optimizer.build(new_model.trainable_variables)

new_model.load_weights('models/' + MODEL_NAME, + '.h5')

In [ ]:
history = new_model.fit(
    train_ds,
    epochs = 3,
    validation_data = val_ds,
    callbacks=[cb],
)

In [ ]:
create_metrics(history, cb, MODEL_NAME)

In [ ]:
transformer.save_weights('models/' + MODEL_NAME, + '.h5')

In [ ]:
#@ Deprecated

# # Save model
# transformer.save('model_en_es_0505_20')
# reconstructed_model = keras.models.load_model("model_en_es_0505_20", custom_objects={ 'CustomSchedule' : CustomSchedule, 'masked_loss': masked_loss, 'masked_acc': masked_acc })

# np.testing.assert_allclose(
#     transformer.predict(val_ds), reconstructed_model.predict(val_ds)
# )

# history = reconstructed_model.fit(
#     train_ds,
#     epochs = 3,
#     validation_data = val_ds,
#     callbacks=[cb],
# )